# 1) Sample the parameter space

From the set of parameters estimated using multiple patch recordings (1 somatic, 2 apical dendrites), we simulate random perturbations using Latin Hypercube Sampling (LHS).

The random parameters are saved in the `config/params/random.csv` and will be used to assess the fitting performance of different feature sets.

In [ ]:
%load_ext autoreload
%autoreload

from pyDOE import lhs
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import json
from pathlib import Path
%matplotlib notebook

In [ ]:
save_params = False

In [ ]:
# load release params
release_params_file = 'config/parameters_release.json'

all_release_params = {}
with open(release_params_file, 'r') as f:
    data = json.load(f)
    
    for prm in data:
        all_release_params[f"{prm['param_name']}.{prm['sectionlist']}"] = prm["value"]

In [ ]:
all_release_params

In [ ]:
# load unfrozen params
params_file = 'config/parameters.json'

params = {}
with open(params_file, 'r') as f:
    data = json.load(f)
    
    for prm in data:
        if "bounds" in prm:
            params[f"{prm['param_name']}.{prm['sectionlist']}"] = prm["bounds"]

In [ ]:
params

In [ ]:
params_to_sample = {}
for k, v in all_release_params.items():
    if k in params.keys():
        params_to_sample[k] = v

In [ ]:
n_params = len(params_to_sample)
n_samples = 100  # number of random samples 
lim_dev = 0.2  # limits for each param are: [release_val-lim_dev*release_val, release_val+lim_dev*release_val] 

In [ ]:
n_params

In [ ]:
params_to_sample

### Latin hypercube sampling (normal 0-1)

In [ ]:
random_samples = lhs(n_params, samples=n_samples)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(random_samples[:, 5], random_samples[:, 19], random_samples[:, 3], '*')
ax.set_xlabel('P1', fontsize=20)
ax.set_ylabel('P2', fontsize=20)
ax.set_zlabel('P3', fontsize=20)

### Compute parameters limits

In [ ]:
param_lims = {}

for par, val in release_params.items():
    dev = lim_dev * val
    lims = [val - dev, val + dev]
    
    # check values based on param configs
    for param in param_configs:
        name, loc = par.split('.')
        if param['param_name'] == name and param['sectionlist'] == loc:
            if 'bounds' in param:
                if lims[0] < param['bounds'][0]:
                    lims[0] = param['bounds'][0]
                    print(f'Param {par} changed lower bound')                    
                if lims[1] > param['bounds'][1]:
                    lims[1] = param['bounds'][1]
                    print(f'Param {par} changed upper bound')
    param_lims[par] = lims

In [ ]:
scaled_samples = np.zeros_like(random_samples)

for i, sample in enumerate(random_samples):
    for j, (par, parlim) in enumerate(param_lims.items()):
        scaled_samples[i, j] = (parlim[1] - parlim[0]) * sample[j] + parlim[0] 

In [ ]:
for sample in scaled_samples:
    for j, (par, parlim) in enumerate(param_lims.items()):
        assert parlim[0] < sample[j] < parlim[1] 

In [ ]:
# creata dataframe
df_release = pd.DataFrame(data=release_params, index=['release'])
df_random = pd.DataFrame(data=scaled_samples, columns=param_lims.keys(), 
                         index=[f'random_{i}' for i in range(n_samples)])

In [ ]:
plt.figure()
ax = sns.boxplot(data=df_random, orient='horizontal')
fig = ax.get_figure()
fig.subplots_adjust(left=0.4)

### Save random and releas parameters to csv

In [ ]:
if save_params:
    params_folder = Path('config/params')

    if not params_folder.is_dir():
        os.makedirs(params_folder)

    release_params_file = params_folder / 'release.csv'
    random_params_file = params_folder / 'random.csv'

    df_release.to_csv(release_params_file, index_label='index')
    df_random.to_csv(random_params_file, index_label='index')    